<a href="https://colab.research.google.com/github/Samuel-wei/yolov5/blob/master/detectTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# utf-8
"""
    Copyright sharebee.cn, Inc, All Rights Reserved
    Author: Samuel
"""
"""
    Reference: https://www.curiousily.com/posts/object-detection-on-custom\
    -dataset-with-yolo-v5-using-pytorch-and-python/
"""
from pathlib import Path
from tqdm import tqdm
import numpy as np
import json
import urllib
import PIL.Image as Image
import cv2
import torch
import torchvision
from IPython.display import display
from sklearn.model_selection import train_test_split

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

#%matplotlib inline
#%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10

np.random.seed(42)


# Each line in the dataset file contains a JSON object. Let's create a list of all annotations

In [ ]:


clothing = []
with open('clothing.json') as f:
    for line in f:
        clothing.append(json.loads(line))
# Here's an example annotation:
print(clothing[0])

for c in clothing:
    if len(c['annotation']) > 1:
        display(c)

# all unique categories
categoris = []
for c in clothing:
    for a in c['annotation']:
        categories.extend(a['label'])
categories = list(set(categories))
categories.sort()
print(categories)

# split the data into a training and validation set
train_clothing, val_clothing = train_test_split(clothing, test_size=0.1)
len(train_clothing), len(val_clothing)

# an image from dataset, start by downloading it
row = train_clothing[10]
img = urllib.request.urlopen(row['content'])
img = Image.open(img)
img = img.convert('RGB')

img.save('demo_image.jpeg', 'JPEG')

print(row)

# OpenCV to read the image
img = cv2.cvtColor(cv2.imread(f'demo_image.jpeg'), cv2.COLOR_BGR2BGRA)
print(img.shape)

# add the bounding box on top of the image along with the label
for a in row['annotation']:
    for label in a['label']:

        w = a['imageWidth']
        h = a['imageHeight']

        points = a['points']
        p1, p2 = points

        x1, y1 = p1['x'] * w, p1['y'] * h
        x2, y2 = p2['x'] * w, p2['y'] * h

        cv2.rectangle(
            img,
            (int(x1), int(y2)),
            (int(x2), int(y2)),
            color=(0, 255, 0),
            thickness=2
        )

        ((label_width), (label_height), _) = cv2.getTextSize(
            label,
            fontFace=cv2.FONT_HERSHEY_PLAIN,
            fontScale=1.75,
            thickness=2
        )

        cv2.rectangle(
            img,
            (int(x1), int(y1)),
            (int(x1 + label_width + label+width * 0.05), int(y1 + label_height + label_height * 0.025)),
            color=(0, 255, 0),
            thickness=cv2.FILLED

        )

        cv2.putText(
            img,
            label,
            org=(int(x1), int(y1 + label_height + label_height * 0.025)), # bottom left
            fontFace=cv2.FONT_HERSHEY_PLAIN,
            fontScale=1.75,
            color=(255, 255, 255),
            thickness=2
        )
# result:
plt.imshow(img)
plt.axis('off');

"""
    One txt with labels file per image
    One row per object
    Each row contains: class_index bbox_x_center bbox_y_center bbox_width bbox_height
    Box coordinates must be normalized between 0 and 1
"""
def create_dataset(clothing, categories, dataset_type):
    images_path = Path(f'clothing/labels/{dataset_type}')
    labels_path.mkdir(parents=True, exist_ok=True)

    for img_id, row in enumerate(tqdm(clothing)):
        image_name = f"{img_id}.jpeg"

        img = urllib.request.urlopen(row['content'])
        img = Image.open(img)
        img = img.convert('RGB')

        img.save(str(images_path / image_name), 'JPEG')

        label_name = f"{img_id}.txt"

        with (labels_path / label_name).open(mode='w') as label_file:
            for a in row['annotation']:
                for label in a['label']:
                    category_idx = categories.index(label)
                    points = a['points']
                    p1, p2 = points
                    x1, y1 = p1['x'], p1['y']
                    x2, y2 = p2['x'], p2['y']

                    bbox_width = x2 - x1
                    bbox_height = y2 - y1

                    label_file.write(
                        f"{category_idx} {x1 + bbox_width / 2} {y1 + bbox_height / 2} {bbox_width} {bbox_height}\n"
                    )
create_dataset(train_clothing, categories, 'train')
create_dataset(val_clothing, categories, 'val')














